# Analyze Historical Calendar Events

In [63]:
import csv
import json
import os
import sys

from datetime import datetime, timedelta
from pytz import timezone

import pandas as pd

from bokeh.io import curdoc, output_notebook, show
from bokeh.layouts import column
from bokeh.models import HoverTool, Range1d, WheelZoomTool
from bokeh.models.widgets import Panel, Tabs
from bokeh.plotting import figure
output_notebook()

sys.path.append(os.path.abspath('../src'))
from util import parse_dt_to_utc
from gcal.util import get_client
from event import from_gcal_events

Loading BokehJS ...

In [55]:
USERNAME = 'zdfs1121@gmail.com'
START_DT = datetime(2018, 1, 1)
END_DT = datetime(2021, 10, 18)
LOCAL_TZ = timezone('America/Los_Angeles')

START_DT_STR = START_DT.isoformat() + 'Z'
END_DT_STR = END_DT.isoformat() + 'Z'

In [56]:
CACHE_PATH = 'PERSONAL_EVENT_CACHE.csv'

In [57]:
# load events from cache or from gcal

def load_events():
    if not os.path.exists(CACHE_PATH):
        calendar = get_client()

        all_results = []
        req = calendar.events().list(
            calendarId=USERNAME,
            timeMin=START_DT_STR,
            timeMax=END_DT_STR,
            singleEvents=True,
            orderBy='startTime'
        )
        while req:
            res = req.execute()
            all_results += res['items']
            req = calendar.events().list_next(req, res)
            print(all_results[-1].get('start', {}).get('dateTime'))

        headers = set()
        for e in all_results:
            headers |= e.keys()

        with open(CACHE_PATH, 'w+') as fout:
            writer = csv.DictWriter(fout, fieldnames=headers)
            writer.writeheader()
            for e in all_results:
                writer.writerow(e)

    return pd.read_csv(CACHE_PATH)

gcal_events = load_events()

print('Days:', (END_DT - START_DT).days)
print('Evts:', len(gcal_events))
print('Cols:', gcal_events.columns)

Days: 1386
Evts: 1409
Cols: Index(['transparency', 'created', 'htmlLink', 'eventType', 'id',
       'guestsCanInviteOthers', 'status', 'iCalUID', 'start', 'updated',
       'sequence', 'location', 'end', 'reminders', 'conferenceData', 'etag',
       'creator', 'originalStartTime', 'hangoutLink', 'organizer',
       'extendedProperties', 'guestsCanModify', 'privateCopy', 'kind',
       'endTimeUnspecified', 'summary', 'attendees', 'source', 'description',
       'guestsCanSeeOtherGuests', 'recurringEventId'],
      dtype='object')


In [58]:
# transform gcal events to nice event models, and those to dataframe
event_models = from_gcal_events(gcal_events, USERNAME)
events = pd.DataFrame([e.serialize() for e in event_models])

def extract_dt_props(row):
    sdl = parse_dt_to_utc(row.startDtStr).astimezone(LOCAL_TZ)
    edl = parse_dt_to_utc(row.endDtStr).astimezone(LOCAL_TZ)
    day_of_week = int(sdl.strftime('%w'))
    day = datetime.strptime(sdl.strftime('%Y-%m-%d'), '%Y-%m-%d')
    month_start = datetime.strptime(sdl.strftime('%Y-%m-01'), '%Y-%m-%d')
    week_start = day - timedelta(days=(day_of_week))
    return {
        'year': sdl.strftime('%Y'),
        'yearMonth': sdl.strftime('%Y-%m'),
        'monthOfYear': sdl.strftime('%m'),
        'monthStartDt': month_start,
        'monthStart': month_start.strftime('%Y-%m-%d'),
        'dayDt': day,
        'day': sdl.strftime('%Y-%m-%d'),
        'dayOfWeek': day_of_week,
        'yearWeek': sdl.strftime('%Y~%U'),
        'weekStartDt': week_start,
        'weekStart': week_start.strftime('%Y-%m-%d'),
        'weekOfYear': sdl.strftime('%U'),
        'durationSec': (edl - sdl).total_seconds()
    }

events = events.join(pd.DataFrame(extract_dt_props(e) for _, e in events.iterrows()))
events = events[events.durationSec < 86400]
events.head()

,title,startDtStr,endDtStr,location,organizerEmail,isOneOnOne,needsLocation,userAcceptanceStr,userIsOrganizer,userHasConflict,...,monthStartDt,monthStart,dayDt,day,dayOfWeek,yearWeek,weekStartDt,weekStart,weekOfYear,durationSec
1,Flight to San Francisco (UA 5215),2018-01-02T13:55:00+00:00,2018-01-02T18:09:00+00:00,Omaha OMA,unknownorganizer@calendar.google.com,False,False,ACCEPTED,False,True,...,2018-01-01,2018-01-01,2018-01-02,2018-01-02,2,2018~00,2017-12-31,2017-12-31,00,15240.0
2,Pandemic Legacy Season 1,2018-01-11T04:30:00+00:00,2018-01-11T07:00:00+00:00,OHB Mansion,zdfs1121@gmail.com,False,False,ACCEPTED,True,False,...,2018-01-01,2018-01-01,2018-01-10,2018-01-10,3,2018~01,2018-01-07,2018-01-07,01,9000.0
3,Flight to San Francisco (VX 1029),2018-01-11T23:59:00+00:00,2018-01-12T06:50:00+00:00,New York City JFK,unknownorganizer@calendar.google.com,False,False,ACCEPTED,False,False,...,2018-01-01,2018-01-01,2018-01-11,2018-01-11,4,2018~01,2018-01-07,2018-01-07,01,24660.0
5,Drinks,2018-01-18T03:00:00+00:00,2018-01-18T05:00:00+00:00,"Bourbon & Branch, 501 Jones St, San Francisco,...",zdfs1121@gmail.com,False,False,ACCEPTED,True,False,...,2018-01-01,2018-01-01,2018-01-17,2018-01-17,3,2018~02,2018-01-14,2018-01-14,02,7200.0
6,Diplomacy,2018-01-20T20:00:00+00:00,2018-01-21T04:00:00+00:00,OHB Mansion,zdfs1121@gmail.com,False,False,ACCEPTED,True,False,...,2018-01-01,2018-01-01,2018-01-20,2018-01-20,6,2018~02,2018-01-14,2018-01-14,02,28800.0


In [59]:
days = events.groupby(['dayDt', 'day']).agg(meetingCount=('title', 'count'), totalSec=('durationSec', 'sum')).reset_index()
days['totalHr'] = days.totalSec / 3600
weeks = events.groupby(['weekStartDt', 'weekStart']).agg(meetingCount=('title', 'count'), totalSec=('durationSec', 'sum')).reset_index()
weeks['totalHr'] = weeks.totalSec / 3600
months = events.groupby(['monthStartDt', 'monthStart']).agg(meetingCount=('title', 'count'), totalSec=('durationSec', 'sum')).reset_index()
months['totalHr'] = months.totalSec / 3600

In [70]:
tab_info = [('Month', 'Month Start Date', 'monthStartDt', 'monthStart', 30), ('Week', 'Week Start Date', 'weekStartDt', 'weekStart', 7), ('Day', 'Day', 'dayDt', 'day', 1)]
tabs = []
for title, hover_title, dt_field, str_field, days_in_interval in tab_info:
    data = events.groupby([dt_field, str_field]).agg(meetingCount=('title', 'count'), totalSec=('durationSec', 'sum')).reset_index()
    data['totalHr'] = data.totalSec / 3600
    
    zoom_tool = WheelZoomTool(dimensions='width', maintain_focus=False)
    tools = [
        HoverTool(tooltips=[
            (hover_title, f'@{str_field}'),
            ('Number of Meetings', '@meetingCount'),
            ('Hours in Meetings', '@totalHr')
        ]),
        zoom_tool,
        'xpan'
    ]
    x_range = Range1d(min(data[dt_field]), max(data[dt_field]), bounds='auto', min_interval=timedelta(days=days_in_interval * 15))
    
    meetings_plot = figure(title=f'Number of Meetings by {title}', x_range=x_range, x_axis_type='datetime', tools=tools, active_scroll=zoom_tool, plot_width=1600, plot_height=400)
    meetings_plot.line(x=dt_field, y='meetingCount', source=data, line_width=2)
    meetings_plot.toolbar_location = None
    
    hours_plot = figure(title=f'Hours Spent in Meetings by {title}', x_range=x_range, x_axis_type='datetime', tools=tools, active_scroll=zoom_tool, plot_width=1600, plot_height=400)
    hours_plot.line(x=dt_field, y='totalHr', source=data, line_width=2)
    hours_plot.toolbar_location = None
    
    tabs.append(Panel(child=column(meetings_plot, hours_plot), title=title))

show(Tabs(tabs=tabs))